## Initialize

In [5]:

import requests
import pandas as pd
import os
import sys
from datetime import datetime
from ipywidgets import Dropdown
import json
import base64
import requests
from dateutil.parser import parse
import fitz


from PyPDF2 import PdfFileReader
from docx2pdf import convert
import docx
# import docs



sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))
import src.esh_client as esh
from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path

headers = {'Content-Type': 'application/json'}
base_url = get_base_url()

def get_doc_type(filename):
    print(filename)



ModuleNotFoundError: No module named 'docs'

## Choose file folder

In [2]:
folder_path = os.path.join('..')
options = next(os.walk(folder_path))[1]
options.remove('notebooks')
selected = Dropdown(
    options=options,
    description='Select:',
    value = None,
    disabled=False,
)
selected

Dropdown(description='Select:', options=('externDocuments', 'internDocuments'), value=None)

## Take all files from folder

In [25]:
if selected.value:
    payload_list = []
    # Get files path
    current_path = os.path.join(sys.path[0], '..', selected.value)
    # Iterate about the folder with all files
    for filename in os.listdir(current_path):
        docType = ''
        filename_blank = ''
        created_at_string = ''
        changed_at_string = ''
        author = ''
        encoded_img = ''
        encoded_doc = ''
        # Read PDF File and encode to base64
        with open(os.path.join(current_path, filename), 'rb') as f:  # open in readonly mode
            if filename.endswith('pdf'):
                docType = 'PDF'
                filename_blank = filename.replace('.pdf', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')

                # Get file info
                pdf = PdfFileReader(f)
                info = pdf.getDocumentInfo()

                created_at = str(info.creation_date_raw)
                created_at_string = created_at[2:10]
                changed_at = str(info.modification_date_raw)
                changed_at_string = changed_at[2:10]
                print(filename_blank + ' Created at: ' + created_at_string +
                      ' Changed at: ' + changed_at_string)
                if info.author == None:
                    author = 'Not specified'
                else:
                    author = info.author

                # get image
                pdf_file = fitz.open(os.path.join(
                    current_path, filename))
                page = pdf_file[0]
                image_page = page.get_pixmap()
                image_page.shrink(2)
                encoded_string_img = base64.b64encode(image_page.tobytes())
                encoded_img = encoded_string_img.decode('ascii')
                encoded_img = 'data:;base64,' + encoded_img

                
            elif filename.endswith('doc'):
                
                docType = 'DOC'
                filename_blank = filename.replace('.doc', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')
                  
            elif filename.endswith('docx'):
                docType = 'DOCX'
                filename_blank = filename.replace('.docx', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')
                
                                
            elif filename.endswith('xls'):
                docType = 'XLS'
                filename_blank = filename.replace('.xls', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')
                
                                
            elif filename.endswith('xlsx'):
                docType = 'XLSX'
                filename_blank = filename.replace('.xlsx', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')
                
                                
            elif filename.endswith('msg'):
                docType = 'MSG'
                filename_blank = filename.replace('.msg', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')
                
                                
            # Append to JSON
            payload = json.dumps({
                'Document': [
                    {
                        'image': encoded_img,
                        'title': filename_blank,
                        'author': author,
                        'text': encoded_doc,
                        'createdAt': created_at_string,
                        'changedAt': changed_at_string,
                        'docType': docType
                    }
                ]
            }, indent=2
            )
            payload_list.append(payload)
            
else:
    print('No folder selected for files')


086 - EfCCNa - European federation of Critical Care Nursing associations Created at: 20070123 Changed at: 20070123
CHAP(2009)00056-UK-Complaint_Redacted Created at: 20090930 Changed at: 20220920
CHAP(2015)00863 Created at: 20160204 Changed at: 20220926
CHAP(2015)01249 Created at: 20180427 Changed at: 20220926
CHAP(2015)01347 Created at: 20180426 Changed at: 20220926


Xref table not zero-indexed. ID numbers for objects will be corrected.


handbook on implementation of the services directive-gp_eudor_WEB_KM7807096ENC_002 Created at: 20071217 Changed at: 20080121
Implementation Report 2020 Created at: 20200511 Changed at: 20200511
SWD 2012-148 implementation SD Created at: 20120608 Changed at: 20120608
user guide PQD EN Created at: 20200716 Changed at: 20220914


## Post data to server

In [23]:
for element in payload_list:
    response = requests.request("POST", f'{base_url}/v1/data/DocResearchDemoExample', headers=headers, data=element)

    print(response.text)

{"Document":[{"id":"7d107380-6748-11ed-9e5c-88d82e07e733"}]}
{"Document":[{"id":"7d2c3cda-6748-11ed-8de7-88d82e07e733"}]}
{"Document":[{"id":"7d42d50c-6748-11ed-9020-88d82e07e733"}]}
{"Document":[{"id":"7d63e49b-6748-11ed-89f6-88d82e07e733"}]}
{"Document":[{"id":"7d767508-6748-11ed-920d-88d82e07e733"}]}
{"Document":[{"id":"7d86e3d3-6748-11ed-8660-88d82e07e733"}]}
{"Document":[{"id":"7d993f78-6748-11ed-aed1-88d82e07e733"}]}
{"Document":[{"id":"7daadb0b-6748-11ed-a967-88d82e07e733"}]}
{"Document":[{"id":"7dbc13d5-6748-11ed-a0a0-88d82e07e733"}]}
{"Document":[{"id":"7dcf1b3c-6748-11ed-9d4d-88d82e07e733"}]}
{"Document":[{"id":"7de1e231-6748-11ed-be45-88d82e07e733"}]}
{"Document":[{"id":"7df33328-6748-11ed-80a9-88d82e07e733"}]}
{"Document":[{"id":"7e07adfb-6748-11ed-b0ab-88d82e07e733"}]}
{"Document":[{"id":"7e236185-6748-11ed-9490-88d82e07e733"}]}
{"Document":[{"id":"7e3734fe-6748-11ed-8096-88d82e07e733"}]}
{"Document":[{"id":"7e63687e-6748-11ed-8675-88d82e07e733"}]}
{"Document":[{"id":"7e74